In [1]:
import time

from functions import *

In [2]:
split_data_tp1_app = open_file('../data/data_tp1_app.txt')
split_data_tp1_dec = open_file('../data/data_tp1_dec.txt')

split_data_tp2_app = open_file('../data/data_tp2_app.txt')
split_data_tp2_dec = open_file('../data/data_tp2_dec.txt')

split_data_tp3_app = open_file('../data/data_tp3_app.txt')
split_data_tp3_dec = open_file('../data/data_tp3_dec.txt')

In [10]:
def get_h_cross_validation(app_data, h_list, cv, method):
    best_h = [0, 0]
    df = pd.DataFrame(app_data)
    shuffled = df.sample(frac=1)
    cut_dfs = np.array_split(shuffled, cv)  
    for h in h_list:
        sum_error = 0
        for i in range(cv):
            df_cv = cut_dfs[i]
            df_train = df.drop(df_cv.index)
            df_cv = df_cv.values.tolist()
            df_train = df_train.values.tolist()
            count_top_1 = 0
            for line in df_cv:
                if method == "uniform":
                    results = get_dict_uniform_value(line[1:], app_data, h)
                elif method == "gaussian":
                    results = get_dict_gaussian_value(line[1:], app_data, h)
                else:
                    print("The given method: {} is not implemented.\n You must choose between \"uniform\" and \"gaussian\"".format(method))
                    exit()
                temp = max(results.values()) 
                res = [key for key in results if results[key] == temp]
                if res[0] == line[0]:
                    count_top_1 = count_top_1 + 1
            error_rate = count_top_1/len(df_cv)
            sum_error = sum_error + error_rate
            print(count_top_1, error_rate, sum_error)
        print(sum_error/cv)
        print("=============")
        if best_h[1] < sum_error/cv:
            best_h[0] = h
            best_h[1] = sum_error/cv
    return best_h


84 0.84 0.84
79 0.79 1.63
83 0.83 2.46
83 0.83 3.29
78 0.78 4.07
0.8140000000000001
75 0.75 0.75
69 0.69 1.44
74 0.74 2.1799999999999997
75 0.75 2.9299999999999997
65 0.65 3.5799999999999996
0.716
71 0.71 0.71
63 0.63 1.3399999999999999
69 0.69 2.03
70 0.7 2.7299999999999995
62 0.62 3.3499999999999996
0.6699999999999999
70 0.7 0.7
64 0.64 1.3399999999999999
65 0.65 1.9899999999999998
70 0.7 2.6899999999999995
62 0.62 3.3099999999999996
0.6619999999999999
68 0.68 0.68
59 0.59 1.27
58 0.58 1.85
73 0.73 2.58
60 0.6 3.18
0.636
66 0.66 0.66
59 0.59 1.25
56 0.56 1.81
70 0.7 2.51
61 0.61 3.1199999999999997
0.6239999999999999
60 0.6 0.6
55 0.55 1.15
50 0.5 1.65
71 0.71 2.36
62 0.62 2.98
0.596
56 0.56 0.56
54 0.54 1.1
53 0.53 1.6300000000000001
69 0.69 2.3200000000000003
65 0.65 2.97
0.5940000000000001
54 0.54 0.54
51 0.51 1.05
57 0.57 1.62
67 0.67 2.29
65 0.65 2.94
0.588
[0.01, 0.8140000000000001]


In [6]:
def get_uniform_val(app_point, dec_point, h):
    result = 0
    dist = compute_one_euclidian_dist(app_point, dec_point)
    if (dist > -h) & (dist < h):
        result = 1
    return result

def get_top_n_decision_parzen(n, theo_class, dists):
    dists_sorted = sorted(dists.items(), key = lambda kv: kv[1], reverse=True)
    cut_dists =dists_sorted[0:n]
    top_n_result = False
    for dist in cut_dists:
        if theo_class == dist[0]:
           top_n_result =True
    return top_n_result

def get_dict_uniform_value(point, data, h):
    result_dict = {}
    class_num = get_unique_class_num(data)
    COUNT_CLASS = 100
    for one_class in class_num:
        count_uniform = 0
        for app_point in data:
            if app_point[0] == one_class:
                count_uniform = count_uniform + get_uniform_val(app_point[1:], point, h)
        result_dict[one_class] = count_uniform/COUNT_CLASS
    return result_dict

def update_confusion_matrix(conf_matrix, line_class, scores_dict):
    row_num = int(line_class) - 1
    temp = max(scores_dict.values()) 
    res = [key for key in scores_dict if scores_dict[key] == temp] 
    col_num = int(res[0]) -1
    conf_matrix[row_num, col_num] = conf_matrix[row_num, col_num] + 1

def compute_uniform_parzen(app_data, dec_data, h):
    count_top_1 = 0
    count_top_2 = 0
    classes_num =get_unique_class_num(app_data)
    conf_matrix = np.zeros((len(classes_num), len(classes_num)))
    for line in dec_data:
        results = get_dict_uniform_value(line[1:], app_data, h)
        if get_top_n_decision_parzen(1, line[0], results):
            count_top_1 = count_top_1 + 1
        if get_top_n_decision_parzen(2, line[0], results):
            count_top_2 = count_top_2 + 1
        update_confusion_matrix(conf_matrix, line[0], results)
    conf_matrix = transform_matrix_to_df(conf_matrix, classes_num)
    print_decision_model_result(len(app_data), len(dec_data), count_top_1/len(dec_data), count_top_2/len(dec_data), conf_matrix=conf_matrix)



In [7]:
def get_gaussian_val(app_point, dec_point, h):
    result = 0
    dist = compute_one_euclidian_dist(app_point, dec_point)
    result = (1 / (h * math.sqrt(2 * math.pi)))* math.exp((-(dist)**2)/(2*(h**2)))
    return result

def get_dict_gaussian_value(point, data, h):
    result_dict = {}
    class_num = get_unique_class_num(data)
    COUNT_CLASS = 100
    for one_class in class_num:
        count_uniform = 0
        for app_point in data:
            if app_point[0] == one_class:
                count_uniform = count_uniform + get_gaussian_val(app_point[1:], point, h)
        result_dict[one_class] = count_uniform/COUNT_CLASS
    return result_dict

def compute_gaussian_parzen(app_data, dec_data, h):
    count_top_1 = 0
    count_top_2 = 0
    classes_num =get_unique_class_num(app_data)
    conf_matrix = np.zeros((len(classes_num), len(classes_num)))
    for line in dec_data:
        results = get_dict_gaussian_value(line[1:], app_data, h)
        if get_top_n_decision_parzen(1, line[0], results):
            count_top_1 = count_top_1 + 1
        if get_top_n_decision_parzen(2, line[0], results):
            count_top_2 = count_top_2 + 1
        update_confusion_matrix(conf_matrix, line[0], results)
    conf_matrix = transform_matrix_to_df(conf_matrix, classes_num)
    print_decision_model_result(len(app_data), len(dec_data), count_top_1/len(dec_data), count_top_2/len(dec_data), conf_matrix=conf_matrix)


## Question  1 
### noyau uniforme

In [5]:
h = 0.5
print("First Data TP1")
start_time = time.time()
compute_uniform_parzen(split_data_tp1_app, split_data_tp1_dec, h)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP2")
start_time = time.time()
compute_uniform_parzen(split_data_tp2_app, split_data_tp2_dec, h)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP3")
start_time = time.time()
compute_uniform_parzen(split_data_tp3_app, split_data_tp3_dec, h)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

First Data TP1
	Results :
	----------------
	Number of elements for the learning step :  500
	Number of elements for the decision step :  500
	----------------

	Top results :
	----------------
	Top 1 rate :  0.642
	Top 2 rate :  0.978
	----------------

	Confusion matrix :
	----------------
╒════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │
╞════╪═════╪═════╪═════╪═════╪═════╡
│  1 │  36 │  63 │   0 │   0 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  2 │  40 │  56 │   0 │   0 │   4 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  3 │   0 │   0 │  95 │   5 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  4 │   0 │   0 │  57 │  39 │   4 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  5 │   1 │   0 │   0 │   4 │  95 │
╘════╧═════╧═════╧═════╧═════╧═════╛
	----------------
	Execution time : 0.3470134735107422 seconds


First Data TP2
	Results :
	----------------
	Number of elements for the learning step :  500
	Number of elements for the decision step :  500
	----------------



### Noyau gaussien

In [ ]:
h = 0.5
print("First Data TP1")
start_time = time.time()
compute_gaussian_parzen(split_data_tp1_app, split_data_tp1_dec, h)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP2")
start_time = time.time()
compute_gaussian_parzen(split_data_tp2_app, split_data_tp2_dec, h)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP3")
start_time = time.time()
compute_gaussian_parzen(split_data_tp3_app, split_data_tp3_dec, h)
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

## Question 2
### Noyau uniforme avec cross-validation

In [8]:
h_list = np.arange(0.2, 2, 0.1)
print(h_list)
print("First Data TP1")
start_time = time.time()
best_h = get_h_cross_validation(split_data_tp1_app, h_list, 5, "uniform")
compute_uniform_parzen(split_data_tp1_app, split_data_tp1_dec, best_h[0])
print("\tThe best h used is {} with a error rate = {}".format(best_h[0], best_h[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP2")
start_time = time.time()
best_h = get_h_cross_validation(split_data_tp2_app, h_list, 5, "uniform")
compute_uniform_parzen(split_data_tp2_app, split_data_tp2_dec, best_h[0])
print("\tThe best h used is {} with a error rate = {}".format(best_h[0], best_h[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP3")
start_time = time.time()
best_h = get_h_cross_validation(split_data_tp3_app, h_list, 5, "uniform")
compute_uniform_parzen(split_data_tp3_app, split_data_tp3_dec, best_h[0])
print("\tThe best h used is {} with a error rate = {}".format(best_h[0], best_h[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09]
First Data TP1
0.8860000000000001
0.828
0.7979999999999999
0.788
0.7859999999999999
0.776
0.748
0.726
0.7380000000000001
	Results :
	----------------
	Number of elements for the learning step :  500
	Number of elements for the decision step :  500
	----------------

	Top results :
	----------------
	Top 1 rate :  0.338
	Top 2 rate :  0.57
	----------------

	Confusion matrix :
	----------------
╒════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │
╞════╪═════╪═════╪═════╪═════╪═════╡
│  1 │  78 │  21 │   0 │   0 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  2 │  86 │  13 │   0 │   0 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  3 │  49 │   0 │  32 │  19 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  4 │  54 │   0 │  29 │  17 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  5 │  67 │   0 │   0 │   4 │  29 │
╘════╧═════╧═════╧═════╧═════╧═════╛
	----------------
	The best h used is 0.01 with a error rate = 0.886000000

### Noyau gaussien avec cross-validation

In [ ]:
h_list = np.arange(0.1, 2, 0.1)
print("First Data TP1")
start_time = time.time()
best_h = get_h_cross_validation(split_data_tp1_app, h_list, 5, "gaussian")
compute_gaussian_parzen(split_data_tp1_app, split_data_tp1_dec, best_h[0])
print("\tThe best k used is {} with a error rate = {}".format(best_h[0], best_h[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP2")
start_time = time.time()
best_h = get_h_cross_validation(split_data_tp2_app, h_list, 5, "gaussian")
compute_gaussian_parzen(split_data_tp2_app, split_data_tp2_dec, best_k[0])
print("\tThe best k used is {} with a error rate = {}".format(best_k[0], best_k[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP3")
start_time = time.time()
best_h = get_h_cross_validation(split_data_tp3_app, h_list, 5, "gaussian")
compute_gaussian_parzen(split_data_tp3_app, split_data_tp3_dec, best_k[0])
print("\tThe best k used is {} with a error rate = {}".format(best_k[0], best_k[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")